### Making filepaths for protocol and data

In [59]:
import pandas as pd
import numpy as np
import seaborn as sns
import os

fp_protocol = "Simulated_folder\\201_20240305_right_sine_b.csv"

rootdir = "..\Simulated_folder"


def make_filepaths(rootdir):
    fp_protocol = [
        os.path.join(subdirs, file)
        for subdirs, dirs, files in os.walk(rootdir)
        for file in files
        if (file.endswith(".csv")) & ("_sine_" in file)
    ][0]
    fp_recording = [
        os.path.join(subdirs, file)
        for subdirs, dirs, files in os.walk(rootdir)
        for file in files
        if (file.endswith("data.csv")) & ("Sequence_" in subdirs)
    ]
    return fp_protocol, fp_recording


fp_protocol, fp_recording = make_filepaths(rootdir)

print(fp_protocol, fp_recording)

.\Simulated_folder\201_20240305_right_sine_b.csv ['.\\Simulated_folder\\Experiment_0001\\Sequence_0001\\data.csv', '.\\Simulated_folder\\Experiment_0001\\Sequence_0002\\data.csv', '.\\Simulated_folder\\Experiment_0001\\Sequence_0003\\data.csv', '.\\Simulated_folder\\Experiment_0001\\Sequence_0004\\data.csv']


### Making protocol dataframes

I split them into one with description variables and one with the timecourse, since they have different column numbers.

In [60]:
def make_protocol_dfs(fp_protocol):
    protocol_vars_df = pd.read_csv(
        fp_protocol,
        skiprows=np.arange(12, 10e6),
        delimiter=";",
        names=["Var", "Val 1", "Val 2"],
        usecols=[i for i in range(3)],
    )
    protocol_timecourse_df = pd.read_csv(
        fp_protocol, skiprows=np.arange(0, 12), delimiter=";"
    )
    return protocol_vars_df, protocol_timecourse_df


protocol_vars_df, protocol_timecourse_df = make_protocol_dfs(fp_protocol)

protocol_vars_df

,Var,Val 1,Val 2
0,LR.exp,civibe_201,NaN
1,Date,09.11.2023,NaN
2,Author(s),Hannah S. Heinrichs,NaN
3,Photoreceptors,CIE tooolbox,NaN
4,Calibration,Source,20230911.0
5,Version,1,0.0
6,NaN,NaN,NaN
7,Sampling time [ms],33,NaN
8,Start delay [s],0,0.0
9,Temperature aquisition interval [tick],20,NaN


In [61]:
protocol_timecourse_df

,NumSample,Label L,LED L1,LED L2,LED L3,LED L4,LED L5,LED L6,L L,M L,...,LED R1,LED R2,LED R3,LED R4,LED R5,LED R6,L R,M R,S R,g R
0,0,dark,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0
1,7273,dark,0,0,0,0,0,0,0,0,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,50,50,50,50
2,0,dark,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0
3,1,dark,0,0,0,0,0,0,0,0,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,100,100,100,0
4,1,dark,0,0,0,0,0,0,0,0,...,0.537161,0.515792,0.462014,0.525009,0.537161,0.475807,100,100,100,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3847,1,dark,0,0,0,0,0,0,0,0,...,0.451295,0.451295,0.451295,0.451295,0.451295,0.451295,100,100,100,100
3848,1,dark,0,0,0,0,0,0,0,0,...,0.463177,0.463177,0.463177,0.463177,0.463177,0.463177,100,100,100,100
3849,1,dark,0,0,0,0,0,0,0,0,...,0.475311,0.475311,0.475311,0.475311,0.475311,0.475311,100,100,100,100
3850,1,dark,0,0,0,0,0,0,0,0,...,0.487613,0.487613,0.487613,0.487613,0.487613,0.487613,100,100,100,100


### Making experiment dataframe

I pull all data files from sequences and concatenate them into one experiment df. I add a column specifying which trial (or adaptation) the specific row belongs to.

In [62]:
def make_experiment_df(fp_recording):

    dfs = []
    for i, filepath in enumerate(fp_recording):
        if i == 0:
            data_df = pd.read_csv(filepath, delimiter=";")
            data_df["Trial number"] = ["Adaptation"] * len(data_df)
            dfs.append(data_df)
        else:
            data_df = pd.read_csv(filepath, delimiter=";")
            data_df["Trial number"] = [i] * len(data_df)
            dfs.append(data_df)

    experiment_df = pd.concat(dfs)
    return experiment_df


experiment_df = make_experiment_df(fp_recording)
experiment_df

,Overall time Sec,Sequence time Sec,Experiment state,Sequence index,Sequences count,Excitation index,Excitation label - Left,Excitation label - Right,Left - Is found,Left - Size Mm,...,Right - Is found,Right - Size Mm,Right - Area Mm,Right - RadiusA Px,Right - RadiusB Px,Right - PosX Px,Right - PoxY Px,Right - Distance from focus,Right - Leds temp,Trial number
0,0.013,0.013,Active,1,26,2,baseline,dark,False,NaN,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.1284,Adaptation
1,0.023,0.024,Active,1,26,2,baseline,dark,True,7.77159,...,True,7.40274,43.04020,152.75288,176.10225,339.23764,618.92163,108.15797,NaN,Adaptation
2,0.038,0.039,Active,1,26,2,baseline,dark,True,7.78543,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Adaptation
3,0.045,0.046,Active,1,26,2,baseline,dark,False,NaN,...,True,7.39352,42.93307,152.29980,176.18649,337.55472,618.94867,109.30186,NaN,Adaptation
4,0.074,0.075,Active,1,26,2,baseline,dark,True,7.79610,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Adaptation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,299.857,19.353,Active,4,26,464,baseline,dark,True,5.58578,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1074,299.878,19.374,Active,4,26,464,baseline,dark,False,NaN,...,True,5.38350,22.76246,109.74530,129.63231,299.84067,565.17151,114.43202,NaN,3
1075,299.893,19.389,Active,4,26,464,baseline,dark,True,5.60921,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1076,299.917,19.413,Active,4,26,464,baseline,dark,False,NaN,...,True,5.39565,22.86530,109.95078,129.97461,299.95258,564.57306,114.18256,NaN,3
